# End Child Poverty Act

A simulation of the End Child Poverty Act and its possible effects (TODO: make a nice paragraph describing what we're doing here)

In [3]:
from openfisca_us import Microsimulation, IndividualSim

In [4]:
from openfisca_us.model_api import *
from openfisca_us.tools.baseline_variables import baseline_variables

REPLACED_PROGRAMS = ["ctc", "eitc"]

CHILD_AMOUNT = 393
CHILD_AGE_LIMIT = 18
ADULT_DEPENDENT_CREDIT = 600
FILER_CREDIT_SINGLE = 600
FILER_CREDIT_MARRIED = 1200
FILER_CREDIT_SINGLE_PHASE_OUT_START = 20_000
FILER_CREDIT_MARRIED_PHASE_OUT_START = 40_000
FILER_CREDIT_PHASE_OUT_RATE = 0.05
FILER_CREDIT_MIN_AGE = 19
FILER_CREDIT_MAX_AGE = 64

class ecpa(Variable):
    value_type = float
    entity = TaxUnit
    definition_period = YEAR
    label = "End Child Poverty Act"

    def formula(tax_unit, period, parameters):
        person = tax_unit.members
        # Child allowance.
        qualifies_for_child_allowance = person("age", period) <= CHILD_AGE_LIMIT
        child_allowance = CHILD_AMOUNT * tax_unit.sum(qualifies_for_child_allowance) * 12
        # Adult dependent credit.
        adult_dependent = person("is_tax_unit_dependent", period) & ~qualifies_for_child_allowance
        adult_dependent_credit = ADULT_DEPENDENT_CREDIT * tax_unit.sum(adult_dependent)
        # Filer credit.
        # Define eligibility based on age.
        age_head = tax_unit("age_head", period)
        age_spouse = tax_unit("age_spouse", period)
        head_qualifies = (age_head >= FILER_CREDIT_MIN_AGE) & (age_head <= FILER_CREDIT_MAX_AGE)
        spouse_qualifies = (age_spouse >= FILER_CREDIT_MIN_AGE) & (age_spouse <= FILER_CREDIT_MAX_AGE)
        filer_credit_eligible = head_qualifies | spouse_qualifies
        # Get maximum amount.
        filing_status = tax_unit("filing_status", period)
        is_married = filing_status == filing_status.possible_values.JOINT
        max_filer_credit = where(is_married, FILER_CREDIT_MARRIED, FILER_CREDIT_SINGLE)
        # Phase out.
        agi = tax_unit("adjusted_gross_income", period)
        phase_out_start = where(is_married, FILER_CREDIT_MARRIED_PHASE_OUT_START, FILER_CREDIT_SINGLE_PHASE_OUT_START)
        excess = max_(agi - phase_out_start, 0)
        reduction = excess * FILER_CREDIT_PHASE_OUT_RATE
        # Compute final amount.
        filer_credit = filer_credit_eligible * max_(max_filer_credit - reduction, 0)
        # Return all three components.
        return child_allowance + adult_dependent_credit + filer_credit

class spm_unit_net_income(Variable):
    def formula(spm_unit, period, parameters):
        original = baseline_variables["spm_unit_net_income"].formula(spm_unit, period, parameters)
        return original + add(spm_unit, period, ["ecpa"])


class add_ecpa(Reform):
    def apply(self):
        for program in REPLACED_PROGRAMS:
            self.neutralize_variable(program)
        self.add_variable(ecpa)
        self.update_variable(spm_unit_net_income)

In [5]:
def make_tax(adults, children, reform):
    if reform is None:
        sim = IndividualSim(year=2022)
    else:
        sim = IndividualSim(reform, year=2022)
    
    sim.add_person(name="head", age=25)
    members = ["head"]
    if adults == 2:
        sim.add_person(name="spouse", age=25)
        members += ["spouse"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=6)
        members += [child]
    sim.add_tax_unit(name="tax_unit", members=members, premium_tax_credit=0)
    sim.add_spm_unit(name="spm_unit", members=members, snap_emergency_allotment=0)
    sim.add_household(name="household", members=members, state_code="MA")

    sim.vary("employment_income", max=100_000, step=100)

    employment_income = sim.calc("employment_income")[0]
    spm_unit_net_income = sim.calc("spm_unit_net_income")[0].round()
    mtr = 1 - sim.deriv(
        "spm_unit_net_income", "employment_income", wrt_target="head"
    )
    
    return pd.DataFrame(
        dict(
            employment_income=employment_income,
            spm_unit_net_income=spm_unit_net_income,
            mtr=mtr,
            adults=adults,
            children=str(children),
            scenario="Baseline" if reform is None else "ECPA",
        )
    )

In [6]:
import pandas as pd

l = []
for adults in range(1, 3):
    for children in range(0, 4):
        for reform in [None, add_ecpa]:
            l.append(make_tax(adults, children, reform))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    spm_unit_net_income="Net income",
    mtr="Marginal tax rate",
    adults="Adults",
    children="Children",
    scenario="Scenario"
)
df

KeyboardInterrupt: 

In [ ]:
wide = df.pivot_table(index=["adults", "children", "employment_income"], columns="scenario", values=["mtr", "spm_unit_net_income"]).reset_index()

wide["diff"] = wide["spm_unit_net_income"]["ECPA"] - wide["spm_unit_net_income"]["Baseline"]
wide["mtr_diff"] = wide["mtr"]["ECPA"] - wide["mtr"]["Baseline"]
wide

In [ ]:
import plotly.express as px

from ubicenter import format_fig

fig = px.line(
    wide,
    "employment_income",
    "diff",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Difference between Baseline vs. ECPA",
)

fig

In [ ]:
px.line(df, x="employment_income", y="diff")

In [ ]:
fig = px.line(
    df,
    x="employment_income",
    y=["mtr_baseline", "mtr_ecpa"]
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat=".1%",
    yaxis_range=[-1, 1],
)
fig.show()

In [ ]:
px.line(df, x="employment_income", y="mtr_diff")

## Microsimulation

In [ ]:
from policyengine.countries import US

us = US()

mb, mr = us._get_microsimulations(add_ecpa)

In [ ]:
res = us.population_reform(add_ecpa)

In [ ]:
res.keys()

In [ ]:
import plotly.graph_objects as go

go.Figure(data=res["rel_income_decile_chart"]["data"], layout=res["rel_income_decile_chart"]["layout"])

In [ ]:
go.Figure(data=res["intra_income_decile_chart"]["data"], layout=res["intra_income_decile_chart"]["layout"])

In [ ]:
go.Figure(data=res["poverty_chart"]["data"], layout=res["poverty_chart"]["layout"])

In [ ]:
in_poverty = mr.calc("spm_unit_is_in_spm_poverty", map_to="person")
child = mb.calc("is_child")
child_poverty = (in_poverty & child).sum() / child.sum()

In [ ]:
in_poverty